# 文件和异常
* 文件
* 异常
* 存储数据
* 测试

# Python 文件操作与 pathlib 包使用指南

## 一、传统文件操作

### 1. 文件打开模式
Python 使用 `open()` 函数操作文件，支持多种模式：

| 模式  | 描述            |
|-----|---------------|
| 'r' | 只读（默认）        |
| 'w' | 写入（会覆盖）       |
| 'x' | 独占创建（文件存在则失败） |
| 'a' | 追加            |
| 'b' | 二进制模式         |
| 't' | 文本模式（默认）      |
| '+' | 读写模式          |

### 2. 基本文件操作示例

```python
# 写入文件
with open('example.txt', 'w', encoding='utf-8') as f:
    f.write('第一行内容\n')
    f.write('第二行内容\n')

# 读取文件
with open('example.txt', 'r', encoding='utf-8') as f:
    content = f.read()  # 读取全部内容
    # 或使用 f.readlines() 读取所有行
    # 或直接迭代 f 逐行读取

# 追加内容
with open('example.txt', 'a', encoding='utf-8') as f:
    f.write('追加的内容\n')
```

### 3. 文件系统操作（os 模块）

```python
import os

# 检查文件/目录是否存在
os.path.exists('path')

# 获取文件大小
os.path.getsize('file.txt')

# 重命名文件
os.rename('old.txt', 'new.txt')

# 删除文件
os.remove('file.txt')

# 创建目录
os.mkdir('new_dir')

# 递归创建目录
os.makedirs('path/to/new_dir')
```

## 二、pathlib 模块（Python 3.4+）

`pathlib` 提供了面向对象的文件系统路径操作，比传统的 `os.path` 更直观。

### 1. 基本使用

```python
from pathlib import Path

# 创建Path对象
p = Path('example.txt')  # 相对路径
p = Path('/path/to/file')  # 绝对路径

# 常用属性
p.name  # 文件名 'example.txt'
p.stem  # 不带后缀的文件名 'example'
p.suffix  # 文件后缀 '.txt'
p.parent  # 父目录 Path对象
p.parts  # 路径各部分组成的元组
p.absolute()  # 获取绝对路径

# 路径拼接
new_path = p.parent / 'new_dir' / 'file.txt'
```

### 2. 文件操作

```python
# 读写文件
content = p.read_text(encoding='utf-8')
p.write_text('内容', encoding='utf-8')

# 二进制读写
data = p.read_bytes()
p.write_bytes(b'binary data')

# 检查路径
p.exists()  # 是否存在
p.is_file()  # 是否是文件
p.is_dir()  # 是否是目录

# 创建目录
p.mkdir(exist_ok=True)  # exist_ok=True避免目录已存在时报错

# 遍历目录
for child in p.parent.iterdir():
    print(child.name)

# 递归遍历
for file in p.parent.glob('**/*.txt'):
    print(file)
```

### 3. 高级操作

```python
# 解析路径
p = Path('/path/to/example.txt')
p.resolve()  # 解析符号链接，返回绝对路径

# 修改文件名
new_p = p.with_name('new_name.txt')

# 修改后缀
new_p = p.with_suffix('.csv')

# 获取文件统计信息
stat = p.stat()
print(stat.st_size)  # 文件大小（字节）
print(stat.st_mtime)  # 最后修改时间（时间戳）

# 创建临时文件/目录
from tempfile import TemporaryDirectory
with TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir) / 'temp.txt'
    tmp_path.write_text('临时内容')
```

### 4. 实用案例

#### 案例1：批量重命名文件

```python
from pathlib import Path

folder = Path('images')
for i, file in enumerate(folder.glob('*.jpg')):
    new_name = f'image_{i:03d}.jpg'
    file.rename(file.with_name(new_name))
```

#### 案例2：查找特定文件

```python
from pathlib import Path

project_dir = Path(__file__).parent  # 获取当前文件所在目录

# 查找所有Python文件
py_files = list(project_dir.glob('**/*.py'))

# 查找大于1MB的文件
large_files = [f for f in project_dir.rglob('*')
               if f.is_file() and f.stat().st_size > 1024*1024]
```

#### 案例3：配置文件处理

```python
from pathlib import Path
import json

config_path = Path.home() / '.myapp' / 'config.json'

# 如果配置文件不存在则创建默认配置
if not config_path.exists():
    config_path.parent.mkdir(exist_ok=True)
    default_config = {'theme': 'dark', 'font_size': 12}
    config_path.write_text(json.dumps(default_config))

# 读取配置
config = json.loads(config_path.read_text())
```

## 三、pathlib vs os.path 对比

| 操作    | os.path                       | pathlib                |
|-------|-------------------------------|------------------------|
| 路径拼接  | `os.path.join('dir', 'file')` | `Path('dir') / 'file'` |
| 获取父目录 | `os.path.dirname(path)`       | `path.parent`          |
| 获取文件名 | `os.path.basename(path)`      | `path.name`            |
| 检查存在  | `os.path.exists(path)`        | `path.exists()`        |
| 是否是文件 | `os.path.isfile(path)`        | `path.is_file()`       |
| 获取后缀  | `os.path.splitext(path)[1]`   | `path.suffix`          |

## 四、最佳实践建议

1. **新项目优先使用 pathlib**：代码更简洁易读
2. **处理路径时使用 `/` 运算符**：比字符串拼接更安全
3. **使用 `with` 语句管理文件对象**：确保资源释放
4. **注意文件编码**：明确指定 `encoding='utf-8'`
5. **处理大文件时使用流式读取**：避免内存问题

pathlib 提供了一种更 Pythonic 的方式来处理文件系统路径，使代码更加清晰和易于维护，特别是在需要跨平台兼容的项目中。